In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/My\ Drive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle

import string
import json
import re

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk

from keras.regularizers import l2
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Add, Reshape, LSTM, Concatenate
from keras.models import Sequential
from keras.models import Model
from keras.optimizers import SGD, Adam


%matplotlib inline

In [ ]:
## load in the data
with open('edited5.pkl', 'rb') as f:
  df = pickle.load(f)
df

In [ ]:
dataset = pd.DataFrame(df)
dataset

,user_id,product_id,product_name,skin_type,rating,review,userId,product_idx
0,1,1,Vitamin C Serum,acne,0.9798,moisturize every day things aveeno drugstorebu...,0,0
1,1,2,The Ordinary,acne,0.9798,moisturize every day things aveeno drugstorebu...,0,1
2,1,3,niacinamide,acne,0.9798,moisturize every day things aveeno drugstorebu...,0,2
3,1,4,salicylic acid,acne,0.9798,moisturize every day things aveeno drugstorebu...,0,3
4,1,5,vitamin C,acne,0.9798,moisturize every day things aveeno drugstorebu...,0,4
...,...,...,...,...,...,...,...,...
115243,23122,4,salicylic acid,acne,0.9872,ive used glow tonic impressed formulated prope...,23121,3
115244,23122,5,vitamin C,acne,0.9872,ive used glow tonic impressed formulated prope...,23121,4
115245,23123,15,cerave,acne,-0.8552,cerave cleanser moisturizer treat acne acne sp...,23122,14
115246,23123,4,salicylic acid,acne,-0.8552,cerave cleanser moisturizer treat acne acne sp...,23122,3


In [ ]:
def lemmatize_stemming(text):
    return (WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
clean_docs = dataset['review'].map(preprocess)
clean_docs[:10]

0    [moisturize, things, aveeno, drugstorebudget, ...
1    [moisturize, things, aveeno, drugstorebudget, ...
2    [moisturize, things, aveeno, drugstorebudget, ...
3    [moisturize, things, aveeno, drugstorebudget, ...
4    [moisturize, things, aveeno, drugstorebudget, ...
5    [similar, routines, skin, typeconcerns, go, ye...
6    [similar, routines, skin, typeconcerns, go, ye...
7    [similar, routines, skin, typeconcerns, go, ye...
8    [similar, routines, skin, typeconcerns, go, ye...
9    [similar, routines, skin, typeconcerns, go, ye...
Name: review, dtype: object

TD-IDF

In [ ]:
dictionary = gensim.corpora.Dictionary(clean_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 acid
1 acne
2 aveeno
3 away
4 best
5 cheap
6 clean
7 clear
8 drugstorebudget
9 good
10 little


In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in clean_docs]
bow_corpus[4310]

In [ ]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 22 ("vitamin") appears 1 time.
Word 33 ("cosrx") appears 2 time.
Word 36 ("currently") appears 1 time.
Word 38 ("dehydrate") appears 1 time.
Word 53 ("help") appears 1 time.
Word 66 ("products") appears 1 time.
Word 72 ("similar") appears 1 time.
Word 81 ("want") appears 1 time.
Word 83 ("water") appears 2 time.
Word 111 ("havent") appears 1 time.
Word 119 ("list") appears 1 time.
Word 120 ("micellar") appears 1 time.
Word 124 ("mouth") appears 1 time.
Word 187 ("need") appears 1 time.
Word 189 ("oily") appears 1 time.
Word 204 ("sure") appears 1 time.
Word 246 ("maybe") appears 1 time.
Word 249 ("mucin") appears 2 time.
Word 259 ("snail") appears 2 time.
Word 260 ("start") appears 1 time.
Word 267 ("advance") appears 2 time.
Word 269 ("aquaphor") appears 1 time.
Word 283 ("follow") appears 2 time.
Word 298 ("mask") appears 1 time.
Word 305 ("pack") appears 1 time.
Word 312 ("rise") appears 3 time.
Word 331 ("tone") appears 1 time.
Word 332 ("toner") appears 1 time.
Word 350 ("pee

In [ ]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.24084319000817261),
 (1, 0.15847689172121887),
 (2, 0.1372428665715738),
 (3, 0.4124291162025813),
 (4, 0.14048279090748808),
 (5, 0.10923375283734865),
 (6, 0.6062828652769537),
 (7, 0.224064675800954),
 (8, 0.1195233150428352),
 (9, 0.1539089048277597),
 (10, 0.2065358974420535),
 (11, 0.07678991109127244),
 (12, 0.054325297687065774),
 (13, 0.09869930816149168),
 (14, 0.10804467633750069),
 (15, 0.1730942428803265),
 (16, 0.11246624680273257),
 (17, 0.1790593937037358),
 (18, 0.12078721155335526),
 (19, 0.13675005744258756),
 (20, 0.18982281389294625),
 (21, 0.12890965462773582),
 (22, 0.08359354141191266)]


Running LDA 

```
# This is formatted as code
```



In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=dictionary, passes=3, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.007*"sodium" + 0.007*"extract" + 0.006*"ingredients" + 0.005*"ceramide" + 0.005*"alcohol" + 0.004*"glycerin" + 0.004*"phosphate" + 0.004*"dimethicone" + 0.004*"glycol" + 0.004*"cetearyl"
Topic: 1 Word: 0.007*"ordinary" + 0.004*"solution" + 0.004*"hormonal" + 0.004*"mask" + 0.004*"serum" + 0.003*"vitamin" + 0.003*"facial" + 0.003*"zinc" + 0.003*"glycolic" + 0.003*"niacinamide"
Topic: 2 Word: 0.003*"toner" + 0.003*"moisturizer" + 0.003*"cream" + 0.003*"hydrate" + 0.003*"cleanse" + 0.003*"apply" + 0.003*"moisturize" + 0.003*"sunscreen" + 0.003*"water" + 0.003*"wash"
Topic: 3 Word: 0.003*"start" + 0.003*"help" + 0.003*"wash" + 0.003*"work" + 0.003*"feel" + 0.003*"try" + 0.003*"moisturizer" + 0.003*"dont" + 0.002*"products" + 0.002*"go"
Topic: 4 Word: 0.005*"essence" + 0.005*"cream" + 0.004*"hyaluronic" + 0.004*"cosrx" + 0.004*"toner" + 0.004*"hydrate" + 0.004*"labo" + 0.004*"mask" + 0.004*"hada" + 0.004*"love"


In [ ]:
vecs = []
for i in range(len(clean_docs)):
    top_topics = lda_model_tfidf.get_document_topics(bow_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(5)]
    vecs.append(topic_vec)

In [ ]:
len(vecs)

115248

In [ ]:
pd.to_pickle(vecs, 'topic_vector1.pkl')